# Happiness Source Prediction

I am going to take help of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering. I am going to use it for this classification task.

In [1]:
# imports 

import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

W0419 22:01:47.086790 140089183954816 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Installing BERT's python package.

In [2]:
!pip install bert-tensorflow

    100% |████████████████████████████████| 71kB 3.4MB/s 


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Loading the data into Google colab

In [4]:
from google.colab import files
uploaded = files.upload()

Saving hm_train.csv to hm_train.csv


In [0]:
import io
train_data = pd.read_csv(io.BytesIO(uploaded['hm_train.csv']))

In [6]:
from google.colab import files
uploaded = files.upload()

Saving hm_test.csv to hm_test.csv


In [0]:
test_data =  pd.read_csv(io.BytesIO(uploaded['hm_test.csv']))

In [0]:
lb_predicted_category = LabelEncoder()
train_data["predicted_category_code"] = lb_predicted_category.fit_transform(train_data["predicted_category"])

In [9]:
train_data.head()

,hmid,reflection_period,cleaned_hm,num_sentence,predicted_category,predicted_category_code
0,27673,24h,I went on a successful date with someone I fel...,1,affection,1
1,27674,24h,I was happy when my son got 90% marks in his e...,1,affection,1
2,27675,24h,I went to the gym this morning and did yoga.,1,exercise,4
3,27676,24h,We had a serious talk with some friends of our...,2,bonding,2
4,27677,24h,I went with grandchildren to butterfly display...,1,affection,1


In [10]:
train_data.predicted_category_code.unique()

array([1, 4, 2, 5, 0, 3, 6])

In [0]:
train_data['predicted_category_code'] = train_data['predicted_category_code'].astype(int)

For us, our input data is the 'cleaned_hm' column and our label is the 'predicted_category_code' column.

In [0]:
DATA_COLUMN = 'cleaned_hm'
LABEL_COLUMN = 'predicted_category_code'

# label_list is the list of labels, 
# i.e. affection, achievement, bonding, enjoy_the_moment, leisure, nature, exercise
label_list = [0, 1, 2, 3, 4, 5, 6]

### Data Preprocessing

We'll need to transform our data into a format BERT understands. This involves two steps. First, we create InputExample's using the constructor provided in the BERT library.

text_a is the text we want to classify, which in this case, is the 'cleaned_hm' in our Dataframe.
text_b is used if we're training a model to understand the relationship between sentences (i.e. is text_b a translation of text_a? Is text_b an answer to the question asked by text_a?). This doesn't apply to our task, so we can leave text_b blank.
label is the label for our example, i.e. nature, leisure etc.

In [0]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train_data.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this task
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (which is included in the Python library):

1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "I love nature" --> ['I', 'love', 'nature']
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens
6. Append "index" and "segment" tokens to each input

To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [14]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])

    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

Instructions for updating:
Colocations handled automatically by placer.


W0419 22:03:01.336816 140089183954816 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0419 22:03:03.505524 140089183954816 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [15]:
tokenizer.tokenize("Let's see if the tokenizer is working or not")

['let',
 "'",
 's',
 'see',
 'if',
 'the',
 'token',
 '##izer',
 'is',
 'working',
 'or',
 'not']

Using our tokenizer, we'll call run_classifier.convert_examples_to_features on our InputExamples to convert them into features BERT understands.

In [16]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 60321


I0419 22:03:06.210250 140089183954816 run_classifier.py:774] Writing example 0 of 60321


INFO:tensorflow:*** Example ***


I0419 22:03:06.214629 140089183954816 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: None


...

I0419 22:03:24.387527 140089183954816 run_classifier.py:774] Writing example 40000 of 60321


INFO:tensorflow:Writing example 50000 of 60321


I0419 22:03:29.060139 140089183954816 run_classifier.py:774] Writing example 50000 of 60321


INFO:tensorflow:Writing example 60000 of 60321


I0419 22:03:33.036381 140089183954816 run_classifier.py:774] Writing example 60000 of 60321


### Creating a model

1. Loading the BERT tf hub module again (this time to extract the computation graph). 
2. Creates a single new layer that will be trained to adapt BERT to our classification task (i.e. identifying the source of happiness). 



In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, num_labels):
    """Creates a classification model."""

    bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
    bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs, signature="tokens",as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]
    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for happiness data.
    output_weights = tf.get_variable( "output_weights", [num_labels, hidden_size],
                                     initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

wrapping our model function in a model_fn_builder function that adapts our model to work for training, evaluation, and prediction.

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps, num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(is_predicting, input_ids, 
                                                               input_mask, segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, 
                                                          num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(label_ids, predicted_labels)
                auc = tf.metrics.auc(label_ids, predicted_labels)
                recall = tf.metrics.recall(label_ids, predicted_labels)
                precision = tf.metrics.precision(label_ids, predicted_labels) 
                
                true_pos = tf.metrics.true_positives(label_ids, predicted_labels)
                true_neg = tf.metrics.true_negatives(label_ids, predicted_labels)   
                false_pos = tf.metrics.false_positives(label_ids, predicted_labels)  
                false_neg = tf.metrics.false_negatives(label_ids, predicted_labels)
                
                return {"eval_accuracy": accuracy, "f1_score": f1_score, "auc": auc, 
                        "precision": precision, "recall": recall, "true_positives": true_pos, 
                        "true_negatives": true_neg, "false_positives": false_pos, 
                        "false_negatives": false_neg}

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metrics)
        
        else:
            (predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask,
                                                         segment_ids, label_ids, num_labels)

            predictions = {'probabilities': log_probs, 'labels': predicted_labels}
            
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn

In [0]:
# Compute train and warmup steps from batch size
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 5000
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [0]:
run_config = tf.estimator.RunConfig(
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [22]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

W0419 22:03:33.262340 140089183954816 estimator.py:1760] Using temporary folder as model directory: /tmp/tmp40mtjlkj


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp40mtjlkj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f68a07e4b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0419 22:03:33.266431 140089183954816 estimator.py:201] Using config: {'_model_dir': '/tmp/tmp40mtjlkj', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f68a07e4b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

### Training the model

In [24]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0419 22:04:05.031569 140089183954816 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0419 22:04:09.187640 140089183954816 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving checkpoints for 1885 into /tmp/tmp40mtjlkj/model.ckpt.


I0419 22:32:19.650875 140089183954816 basic_session_run_hooks.py:594] Saving checkpoints for 1885 into /tmp/tmp40mtjlkj/model.ckpt.


INFO:tensorflow:Loss for final step: 0.27776176.


I0419 22:32:31.048805 140089183954816 estimator.py:359] Loss for final step: 0.27776176.


Training took time  0:28:57.709335


Let's write code make prediction on the test data

In [0]:
def getPrediction(in_sentences):
  labels = ['achievement','affection','bonding','enjoy_the_moment','exercise','leisure','nature']
  
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  #return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]
  return [labels[prediction['labels']] for sentence, prediction in zip(in_sentences, predictions)]

In [26]:
predictions = getPrediction(test_data.cleaned_hm)

INFO:tensorflow:Writing example 0 of 40213


I0419 22:32:31.254864 140089183954816 run_classifier.py:774] Writing example 0 of 40213


INFO:tensorflow:*** Example ***


I0419 22:32:31.257920 140089183954816 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: 


...

I0419 22:33:13.246273 140089183954816 saver.py:1270] Restoring parameters from /tmp/tmp40mtjlkj/model.ckpt-1885


INFO:tensorflow:Running local_init_op.


I0419 22:33:14.144381 140089183954816 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0419 22:33:14.252605 140089183954816 session_manager.py:493] Done running local_init_op.


### Creating the submission file

In [0]:
subm = {'hmid': test_data['hmid'],
        'predicted_category': predictions}

In [0]:
submission = pd.DataFrame(subm)

In [0]:
submission.to_csv('submission_happiness_v6.csv', index =False)

In [0]:
from google.colab import files
files.download("submission_happiness_v6.csv")